In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri
import matplotlib.collections

In [4]:
numSeeds = 24
radius = 100
seeds = radius * np.random.random((numSeeds, 2))
print("seeds:\n", seeds)
print("BBox Min:", np.amin(seeds, axis=0),"Bbox Max: ", np.amax(seeds, axis=0))

seeds:
 [[17.13126051 24.44715086]
 [59.97150381 64.39919996]
 [92.58512891 14.38061868]
 [34.02255079 36.46437825]
 [73.78590796 98.01206078]
 [60.15607368 29.99412952]
 [71.55785349 86.92862887]
 [42.91965379 30.67496642]
 [59.01624341 82.70252375]
 [23.9060448  22.05521145]
 [86.11048292  9.09217183]
 [93.80822276 33.6042278 ]
 [77.67816664 18.08417211]
 [ 6.33782741 17.55560609]
 [66.83930717 33.97369974]
 [58.85937516 30.0371319 ]
 [49.97210259 34.06508465]
 [75.58941959 47.91833465]
 [79.74214849 31.97096019]
 [97.93216353 60.39081648]
 [38.55964524 48.31963696]
 [48.3898994  14.00339529]
 [72.69884249 90.36437589]
 [86.24378323 43.49558307]]
BBox Min: [6.33782741 9.09217183] Bbox Max:  [97.93216353 98.01206078]


In [5]:
center = np.mean(seeds, axis=0)
print("Center:", center)
center = np.asarray(center)
# Create coordinates for the corners of the frame
coords = [center+radius*np.array((-1, -1)),center+radius*np.array((+1, -1)),center+radius*np.array((+1, +1)),center+radius*np.array((-1, +1))]

Center: [61.40890032 41.78891938]


In [7]:
def circumcenter( tri):
    """Compute circumcenter and circumradius of a triangle in 2D.
    Uses an extension of the method described here:
    http://www.ics.uci.edu/~eppstein/junkyard/circumcenter.html
    """
    pts = np.asarray([coords[v] for v in tri])
    pts2 = np.dot(pts, pts.T)
    A = np.bmat([[2 * pts2, [[1],[1],[1]]],[[[1, 1, 1, 0]]]])
    b = np.hstack((np.sum(pts * pts, axis=1), [1]))
    x = np.linalg.solve(A, b)
    bary_coords = x[:-1]
    center = np.dot(bary_coords, pts)
    # radius = np.linalg.norm(pts[0] - center) # euclidean distance
    radius = np.sum(np.square(pts[0] - center))  # squared distance
    return (center, radius)

In [8]:
# Create two dicts to store triangle neighbours and circumcircles.
triangles = {}
circles = {}
# Create two CCW triangles for the frame
T1 = (0, 1, 3)
T2 = (2, 3, 1)
triangles[T1] = [T2, None, None]
triangles[T2] = [T1, None, None]

In [9]:
def inCircleFast( tri, p):
    """Check if point p is inside of precomputed circumcircle of tri.
    """
    center, radius = circles[tri]
    return np.sum(np.square(center - p)) <= radius

In [10]:
# Compute circumcenters and circumradius for each triangle
for t in triangles:
    circles[t] = circumcenter(t)